In [1]:
# connect and chage drive to ROOT dir

import os

ROOT = "/content/drive/MyDrive/002_iNeuron/Teaching_BTM/NLP_t_extra/k_train_trial"
os.chdir(ROOT)

assert os.getcwd(), ROOT

In [3]:
try:
    import ktrain
except:
    print("* installing ktrain...")
    !pip install -q ktrain
    print("* installation done! importing ktrain")
    import ktrain

* installing ktrain...
     |████████████████████████████████| 25.3MB 135kB/s 
     |████████████████████████████████| 6.8MB 56.4MB/s 
     |████████████████████████████████| 983kB 45.1MB/s 
     |████████████████████████████████| 266kB 58.1MB/s 
     |████████████████████████████████| 1.3MB 48.7MB/s 
     |████████████████████████████████| 1.1MB 54.0MB/s 
     |████████████████████████████████| 471kB 52.6MB/s 
     |████████████████████████████████| 2.9MB 52.4MB/s 
     |████████████████████████████████| 890kB 52.5MB/s 
ERROR: transformers 3.5.1 has requirement sentencepiece==0.1.91, but you'll have sentencepiece 0.1.94 which is incompatible.
* installation done! importing ktrain


In [4]:
from ktrain import text

[sklearn-dataset 20newsgroup](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html)

In [9]:
# import a dataset: 20newsgroup with limited categories

categories = ['alt.atheism', 
              'soc.religion.christian',
              'comp.graphics', 
              'sci.med']

from sklearn.datasets import fetch_20newsgroups

train_b = fetch_20newsgroups(
    subset='train',
    categories=categories, 
    shuffle=True, 
    random_state=42
    )

test_b = fetch_20newsgroups(
    subset='test',
    categories=categories, 
    shuffle=True, 
    random_state=42
    )

print(f'size of training set: {len(train_b["data"])}')
print(f'size of validation set: {len(test_b["data"])}')
print(f'classes: {train_b.target_names}')

x_train = train_b.data
y_train = train_b.target
x_test = test_b.data
y_test = test_b.target

size of training set: 2257
size of validation set: 1502
classes: ['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']


In [16]:
print(x_train[0])

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
Organization: The City University
Lines: 14

Does anyone know of a good way (standard PC application/PD utility) to
convert tif/img/tga files into LaserJet III format.  We would also like to
do the same, converting to HPGL (HP plotter) files.

Please email any response.

Is this the correct group?

Thanks in advance.  Michael.
-- 
Michael Collier (Programmer)                 The Computer Unit,
Email: M.P.Collier@uk.ac.city                The City University,
Tel: 071 477-8000 x3769                      London,
Fax: 071 477-8565                            EC1V 0HB.



In [17]:
# Preprocess data

trn, val, preproc = text.texts_from_array(x_train=x_train, 
                                          y_train=y_train,
                                          x_test=x_test, 
                                          y_test=y_test,
                                          class_names=train_b.target_names,
                                          preprocess_mode='distilbert',
                                          maxlen=350)


preprocessing train...
language: en
train sequence lengths:
	mean : 308
	95percentile : 837
	99percentile : 1938


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 343
	95percentile : 979
	99percentile : 2562


task: text classification


In [18]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [19]:
model = text.text_classifier('distilbert', 
                             train_data=trn, 
                             preproc=preproc)

Is Multi-Label? False
maxlen is 350



done.


In [20]:
learner = ktrain.get_learner(model, 
                             train_data=trn, 
                             val_data=val, 
                             batch_size=6)

In [21]:
LEARNING_RATE = 3e-5
EPOCHS = 4
learner.fit_onecycle(lr=LEARNING_RATE, epochs=EPOCHS)



begin training using onecycle policy with max lr of 3e-05...
Epoch 1/4
377/377 [==============================] - 122s 303ms/step - loss: 1.0808 - accuracy: 0.5529 - val_loss: 0.2795 - val_accuracy: 0.9148
Epoch 2/4
377/377 [==============================] - 116s 306ms/step - loss: 0.1468 - accuracy: 0.9579 - val_loss: 0.1662 - val_accuracy: 0.9447
Epoch 3/4
377/377 [==============================] - 116s 306ms/step - loss: 0.0672 - accuracy: 0.9805 - val_loss: 0.1407 - val_accuracy: 0.9587
Epoch 4/4
377/377 [==============================] - 116s 307ms/step - loss: 0.0155 - accuracy: 0.9955 - val_loss: 0.1287 - val_accuracy: 0.9627


In [31]:
predictor = ktrain.get_predictor(model, preproc=preproc)

In [32]:
text_in = "There is a problem with my computer monitor's resolution.  Everything is blurry."

predictor.predict(texts=text_in)

'comp.graphics'

In [33]:
predictor.save("trained/predictor_1")

In [35]:


new_predictor = ktrain.load_predictor("trained/predictor_1")
new_predictor.predict(text_in)

'comp.graphics'

In [36]:
new_predictor.predict("I belong to Cristian community")

'soc.religion.christian'

In [37]:
new_predictor.predict("Last Christmas was spent with the lot of enthusiasm")

'soc.religion.christian'

In [38]:
new_predictor.predict("Covid 19 caused international health emergency in 2020")

'sci.med'

In [ ]:
new_predictor.predict("")